[View in Colaboratory](https://colab.research.google.com/github/lycohen/ds_desafio_I/blob/master/Desafio_1_fills.ipynb)

# LAB:   Desafío 1.

## Objetivos

* Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.
* Realizar un análisis descriptivo de las principales variables.
* Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.

El dataset contiene información referida al primer semestre de 2017 de las propiedades comercializables por la empresa:

* Fecha de creación (solo un mes en analisis)
* Tipo de la propiedad (house, apartment, ph )
* Operación del aviso (sell, rent)
* Nombre del lugar
* Nombre del lugar + nombre de sus ‘padres’
* ID de geonames del lugar (si está disponible)
* Latitud,Longitud
* Precio original del aviso
* Moneda original del aviso (ARS, USD)
* Precio del aviso en moneda local (ARS)
* Precio aproximado en USD
* Superficie en m²
* Superficie cubierta en m²
* Precio en USD/m²
* Precio por m²
* N° de piso, si corresponde
* Ambientes
* URL en Properati
* Descripción
* Título
* URL de un thumbnail de la primer foto

## ¿Cómo empezar? Sugerencias
Agreguen toda otra información construida a partir de los datos originales (o incluso
información externa) que consideren relevante y útil para resolver los objetivos planteados.
Aprovechen las herramientas de pandas: gropuby , summation, pivot_tables y otras
aplicaciones y métodos de los DataFrames hacen mucho más simples los cálculos y otras
agregaciones de los datos.
En la presentación de los resultados tengan en cuenta que es altamente probable que la
audiencia no tenga un nivel técnico así que mantengan el lenguaje en un nivel accesible.
En términos generales, recuerden las siguientes sugerencias:
* escribir un pseudocódigo antes de empezar a codear. Suele ser muy útil para darle un esquema y una lógica generales al análisis
* leer la documentación de cualquier tecnología o herramienta de análisis que uses. A veces no hay tutoriales para todo y los documentos y las ayudas son fundamentales para entender el funcionamiento de las herramientas utilizadas
* documentar todos los pasos, transformaciones, comandos y análisis que realices.

## Recursos útiles
* Documentación de la librería GeoPandas
* Github

##Tareas

##1 - ver calidad
* NaN
* vacios
* inconsistencias entre keywords columnas vs keywords url
* monedad $/u$s
* EDA / Media central y outliers
* correlacion
* MNAR/MNCAR etc
* mejor opcion de fill

In [0]:
% matplotlib inline
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
#from sklearn.feature_extraction.text import CountVectorizer
import unidecode
import re
#from unidecode import unidecode #QUITO TILDES


In [0]:
# Cargar la data de properati
df = pd.read_csv('properatti.csv')  # DATOS DE PROPIEDADES EN VENTA PROPERATI


## <font color='red'>Armamos la matriz ampliada y vamos limpiando de a poco .</font> 

In [0]:

###Obtengo 10000 para probar
df2 = df
df.columns


Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [0]:


def sacoAcentos(s):
    return unidecode.unidecode(s)
#cotizacion = getCotizacionUSD('2002-04-03 00:00:00')

# Rastreamos las localidades que tengan 3 o m�s d�gitos seguidos y laa establecemos en None para luego corregirlas.
# Por ejemplo place_name = 'coordenadas 34.255511'
def corrigeCoordEnPlaceName(place_name):
    if re.compile('\d{3,}').findall(str(place_name)):
        return None
    else:
        return place_name
    

def GetAmmenities(description):
    #"Extrae los ammenities del camp descripcin"
    #### Usamos la info extraida del analisis del notebook Desafio I - palabras predictoras
    ###  Aun asi usamos algunas otras "cisnes negros" que no estaban en el top 50 de palabras mas usadas.
    description = str(description).lower()
    description = sacoAcentos(description)
    ammenities_to_extract = ['lavadero', 'pileta_piscina', 'telefono','terraza','nuevo', 'refaccionado', 'AC', 'living_comedor', 'cocina_comedor', 'cocina_integrada', 'parrilla','balcon', 'subte', 'cochera_fija', 'baulera']

    #0
    lavadero = (0, 1)['lavadero' in description] # reemplaza el if else statement 
    #1
    pileta_piscina = (0, 1)['pileta' in description or 'piscina' in description ]
    #2
    terraza = (0, 1)['terraza' in description]
    #3
    nuevo = (0, 1)['nuevo' in description]
    #4
    refaccionado = (0, 1)['refaccionado' in description]
    #5
    AC = (0, 1)['aire acondicionado' in description or 'AC' in description or 'AA' in description ]
    #6
    living_comedor = (0, 1)['living comedor' in description ]
    #7
    cocina_comedor = (0, 1)['cocina comedor' in description ]
    #8
    cocina_integrada = (0, 1)['cocina integrada' in description ]
    #9
    parrilla = (0, 1)['parrilla' in description ]
    #10
    balcon = (0, 1)['balcon' in description or 'balcon' in description ]
    #11
    subte =(0, 1)['subte' in description or 'metro' in description ]
    #12
    cochera_fija = (0, 1)['cochera fija' in description or 'cochera' in description ]
    #13
    baulera = (0, 1)['baulera' in description ]
    telefono = (0, 1)['telefono' in description ]

    extracted_ammenities = pd.Series([lavadero,pileta_piscina, terraza,nuevo, refaccionado, AC, living_comedor, cocina_comedor, cocina_integrada, parrilla,balcon, subte, cochera_fija, baulera,telefono])
    #print("Extracted ammenities:", list(zip(ammenities_to_extract, extracted_ammenities)))
    return extracted_ammenities
# Cargar la data de properati

def getAmbientes(description):
    ### Obtiene la cantidad de ambientes de la descripcion
    description = str(description).lower()
    numbers_as_strings = {
        'un': 1,
        'dos': 2,
        'tres': 3,
        'cuatro': 4,
        'cinco': 5,
        'seis': 6,
        'seis': 7
    }
 
   
    # Con esta regex extraemos los ambientes contemplando los casos de texto y de números.
    result = pd.Series([description]).str.extract("(\d|un|dos|tres|cuatro|cinco|seis)(\s+)(amb)",expand=False)
    # Retorna un Dataframe con el resultado de la extracción, lo obtenemos con sus indices.
    matched_value = result[0][0]
    #print (matched_value)
    
    ambientes = 'SD'
    try:
        # No encontramos la cantidad de ambientes en la descripción
        if np.isnan(result[0][0]):
            ambientes = 'SD'
    except TypeError:
        try:
            # Lo encontramos, intentamos parsearlo como entero.
            ambientes = int(matched_value)
        # Lo encontramos como string, lo mapeamos con un diccionario.
        except ValueError:
            ambientes = numbers_as_strings[matched_value]
    return ambientes


def splitPlaceWithParentNames(place_with_parent_names):
    #### Obtengo placeName de place_with_parent_names para reemplazar el place_name. (hay datos faltantes en place_name)
    ### faltan 24 valores
    splittedLocation = place_with_parent_names.split('|')
    stateName= splittedLocation[2]
    placeName = splittedLocation[3]
    return pd.Series([stateName, placeName])

## <font color='red'>Corrijo la coord 34 en place_name
  .</font> **texto en negrita**

In [0]:
####Remuevo los que tienen la palabra coordenadas en place_name
df['place_name'] = df['place_name'].apply(corrigeCoordEnPlaceName)
# Chequeo de la corrección.
df[df['place_name'] == 'coordenadas 34.255511']

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail


## <font color='red'>Obtengo AMMENITIES .</font> 

In [0]:
####Obtengo   ammenities por item y creo un DF Ampliado con el original + ammenities.
dfAmmenities= df2['description'].apply(GetAmmenities)
#df3.rename(columns={'0':'lavadero', , ,'nuevo', '3':'refaccionado','4': 'AC','5':'living_comedor','6': 'cocina_comedor', '7':'cocina_integrada', '8':'parrilla','9':'balcon','10':'subte','11':'cochera_fija','12':'baulera'})
dfAmpliada = df2.join(dfAmmenities)
dfAmpliada = dfAmpliada.rename(columns={0:'lavadero',1:'pileta',2:'terraza',3:'nuevo',4:'refaccionado',5:'aire',6:'living',7:'cocina_comedor',8:'cocina_integrada',9:'parrilla',10:'balcon',11:'subte',12:'cochera_fija',13:'baulera',14:'telefono'})


## <font color='red'>Imputo ambientes faltantes del title.</font> 

In [0]:
####Obtengo   ambientes por item y creo un DF Ampliado con el original + ambientes ¿Lo  usamos para pisar ambientes NAN?.
dfAmbientes =  pd.DataFrame(  df2['description'].apply(getAmbientes))
dfAmbientes = dfAmbientes.rename(columns={'description':'ambientes_title'})


In [0]:
dfAmpliada['ambientes_title']=dfAmbientes.ambientes_title

In [0]:
##Tengo 21814 rooms para llenar.
dfAmpliada[(dfAmpliada['rooms'].isna()) & (dfAmpliada['ambientes_title']!='SD')].shape[0]

21814

In [0]:
##reemplazo los ambientes del titulo en los nan de rooms
dfAmpliada['rooms'].fillna(dfAmpliada['ambientes_title'],inplace=True)

## <font color='red'>REEMPLAZO PLACE_NAME FALTANTES CON LOS PLACE_NAME DE PLACE_WITH PARENT NAMES</font> 

In [0]:
##Obtengo un DF Auxiliar con el place with parent names partido por cada uno de sus componentes.
#me voy a quedar con place_name y lo voy a reemplazar en el DF Apmpliado
dfUbicacion = pd.DataFrame(dfAmpliada['place_with_parent_names'].apply(splitPlaceWithParentNames))

dfUbicacion =dfUbicacion.rename(columns={0:'filled_state_name',1:'filled_place_name'})
dfAmpliada['filled_place_name'] = dfUbicacion['filled_place_name']
# Completamos los faltantes en la variable place_name con los datos extra�dos.
dfAmpliada['place_name'].fillna(value=dfAmpliada['filled_place_name'], inplace=True)



In [0]:
###esta corregido?
dfAmpliada['place_name'].isna().sum()

0

## <font color='red'>VAMOS CON LAT. Y LONG</font> 

In [0]:
print('cant nulos longitud')
display(dfAmpliada['lon'].isna().sum())

cant nulos longitud


51550

In [0]:
print('cant nulos latitud')
display(dfAmpliada['lat'].isna().sum())

cant nulos latitud


51550

In [0]:
###### Busco completar latitud - Seteo la media de latitud para los de una misma zona.
dflat = dfAmpliada.groupby(['place_with_parent_names'])[['lat']].median()
dflon = dfAmpliada.groupby(['place_with_parent_names'])[['lon']].median()
dflon['place_with_parent_names'] = dflon.index.values
dflat['place_with_parent_names'] = dflat.index.values
dfillLatLong = pd.merge(left= dflat, right=dflon, left_on='place_with_parent_names', right_on='place_with_parent_names' )
dfAmpliada2 = pd.merge(left= dfAmpliada, right=dfillLatLong, left_on='place_with_parent_names', right_on='place_with_parent_names', suffixes='np' )


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'place_with_parent_names' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [0]:
dfAmpliada2.rename( columns={'latp':'lat','lonp':'lon'}, inplace=True) 
#dfAmpliadaV2[dfAmpliadaV2['lat_x'].isna()]


In [0]:
 dfAmpliada2.columns




Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'latn', 'lonn', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail', 'lavadero', 'pileta', 'terraza', 'nuevo',
       'refaccionado', 'aire', 'living', 'cocina_comedor', 'cocina_integrada',
       'parrilla', 'balcon', 'subte', 'cochera_fija', 'baulera', 'telefono',
       'ambientes_title', 'filled_place_name', 'lat', 'lon'],
      dtype='object')

In [0]:
dfAmpliada2.drop(['lat-lon','latn','lonn'], axis=1,inplace=True)


In [0]:
dfAmpliada2['lat'].isna().sum()

838

In [0]:
dfAmpliada2['lon'].isna().sum()

838

In [1]:
dfAmpliada2['lon'].describe()

NameError: ignored

## <font color='red'>BORRO INNECESARIAS.</font> 

In [0]:
dfAmpliada2.drop(['Unnamed: 0','operation','country_name','filled_place_name','floor','expenses','properati_url','image_thumbnail'], axis=1,inplace=True)


In [0]:

# Creamos una pivot table para la media de surface_covered_in_m2 agrupada por place_name y property_type.
flattened_pivot_surface = pd.DataFrame(dfAmpliada.pivot_table(values = 'surface_covered_in_m2', index = ['place_name','property_type'],
                                aggfunc = 'mean').to_records())


 

In [0]:

                                
# Lo mergeamos con el dataframe original.
cleaned_df = dfAmpliada2.merge(flattened_pivot_surface, how='left', on=['place_name','property_type'])

# Imputamos las medias agrupadas y eliminamos las variables redundantes
cleaned_df.surface_covered_in_m2_x.fillna(cleaned_df.surface_covered_in_m2_y, inplace=True)
del cleaned_df['surface_covered_in_m2_y']
cleaned_df['surface_covered_in_m2'] = cleaned_df['surface_covered_in_m2_x']
del cleaned_df['surface_covered_in_m2_x']

cleaned_df.surface_covered_in_m2.isna().sum()


362

In [0]:
cleaned_df.to_csv('properatti.editado.csv', index = False, header = True)

In [0]:
 # Lo mergeamos con el dataframe original.
cleaned_df = dfAmpliada2.merge(flattened_pivot_surface, how='left', on=['place_name','property_type'])

# Imputamos las medias agrupadas y eliminamos las variables redundantes
cleaned_df.surface_covered_in_m2_x.fillna(cleaned_df.surface_covered_in_m2_y, inplace=True)
del cleaned_df['surface_covered_in_m2_y']
cleaned_df['surface_covered_in_m2'] = cleaned_df['surface_covered_in_m2_x']
del cleaned_df['surface_covered_in_m2_x']

cleaned_df.surface_covered_in_m2.isna().sum()

